In [7]:
import pickle
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

data = pd.read_json('generated_data.json')
data

,proj_id,team_size,budget,workload,completion_time,assigned_manager,tasks
0,1,2,10125.05,medium,155,Victoria Dunn,"[{'id': 1, 'description': 'Document integratio..."
1,2,7,10158.91,low,141,Victoria Dunn,"[{'id': 1, 'description': 'Implement integrati..."
2,3,8,12946.33,low,111,Joseph Parker,"[{'id': 1, 'description': 'Implement backend l..."
3,4,2,14865.58,medium,70,George Shannon,"[{'id': 1, 'description': 'Implement user inte..."
4,5,6,11776.35,high,166,Joseph Parker,"[{'id': 1, 'description': 'Document backend lo..."
...,...,...,...,...,...,...,...
995,996,7,14304.52,medium,150,Adam Armstrong,"[{'id': 1, 'description': 'Test user interface..."
996,997,6,11639.45,low,74,Adam Armstrong,"[{'id': 1, 'description': 'Test integration mo..."
997,998,3,11636.96,medium,263,David Johnson,"[{'id': 1, 'description': 'Optimize integratio..."
998,999,7,13117.18,high,177,Henry Carr,"[{'id': 1, 'description': 'Design database sch..."


In [3]:
data.isnull().sum(axis = 0)

proj_id                      1502
team_size                    1502
budget                       1502
workload                     1502
completion_time              1502
assigned_manager             1502
tasks__id                       0
tasks__description              0
tasks__status                   0
tasks__person_assigned          0
tasks__due_date                 0
tasks__estimated_duration       0
dtype: int64

In [10]:
import pandas as pd
import json

# Load JSON data
with open('generated_data.json') as f:
    data = json.load(f)

# Normalize JSON data to flatten nested structure
df = pd.json_normalize(data, 'tasks', ['proj_id', 'team_size', 'budget', 'workload', 'completion_time', 'assigned_manager'])

# Group by project ID and count the number of tasks
task_counts = df.groupby('proj_id').size()

# Drop the 'tasks' column from the DataFrame
df.drop(columns=['tasks'], inplace=True)

# Reset index of task_counts to match proj_id
task_counts = task_counts.reset_index(name='task_count')

# Merge task_counts into the DataFrame on 'proj_id'
df = df.merge(task_counts, on='proj_id')

# Display the updated DataFrame
print(df)


KeyError: "['tasks'] not found in axis"